In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\user\\Desktop\\deep-learning'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir:Path
    trained_model_file_path:Path
    updated_base_model:Path
    training_data:Path
    params_epochs:int
    params_batch_size:int
    params_is_augmentation:bool
    params_image_size:int
    params_learning_rate:int

@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir:Path
    tensorboard_root_log_dir:Path
    checkpoint_model_file_path:Path


In [5]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml,create_directories

In [6]:
os.path.join("artifacts/data_ingestion","Chicken-fecal-images")

'artifacts/data_ingestion\\Chicken-fecal-images'

In [ ]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback(self)->PrepareCallbackConfig:
        config=self.config.prepare_callbacks

        model_ckpt_dir=os.path.dirname(config.checkpoint_model_file_path)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config=PrepareCallbackConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_file_path=Path(config.checkpoint_model_file_path)


        )
    
        return prepare_callback_config
    
    def get_training_config(self)->ModelTrainerConfig:
        training=self.config.training
        prepare_base_model=self.config.prepare_base_model
        params=self.params
        training_data=os.path.join(self.config.data_ingestion.unzip_dir,"Chicken-fecal-images")
        create_directories([
            Path(training.root_dir)
        ])

        training_config=ModelTrainerConfig(
            root_dir=Path(training.root_dir),
            trained_model_file_path=Path(training.trained_model_file_path),
            updated_base_model=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_image_size=params.IMAGE_SIZE,
            params_is_augmentation=params.AUGMENTATION
            params_learning_rate=params.LEARNING_RATE
            

        )
        return training_config
        

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time


In [9]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)


In [10]:
list=[224,224,3]
list[:-1]

[224, 224]

In [11]:
from typing import List

In [12]:
class PrepareCallback:

    def __init__(self,config:PrepareCallbackConfig):
        self.config=config

    @property
    def create_tb_callbacks(self):
        timestamp=time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir=os.path.join(self.config.tensorboard_root_log_dir,
                                        f"tb_logs_at_{timestamp}")
        
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_file_path,
            save_best_only=True
        )
    
    @property

    def get_tb_callback(self):
        return [
            self.create_tb_callbacks,
            self.create_ckpt_callbacks
        ]


In [13]:
class Training:
    
    def __init__(self,config:ModelTrainerConfig):
        self.config=config

    def get_base_model(self):
        self.model=tf.keras.models.load_model(
            self.config.updated_base_model

        )

    def train_valid_generator(self):
        datagenerator_kwargs=dict(
            rescale=1/255.0,
            validation_split=0.2
        )

        dataflow_kwargs=dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation='bilinear'
        )
        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                **datagenerator_kwargs
                
            )

        else:
            train_datagenerator=valid_datagenerator

        self.train_generator=train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs

            
        )

    @staticmethod
    def save_model(model:tf.keras.Model,path:Path):
        model.save(path)


    def train(self,callback_list:List):
        self.steps_per_epoch=self.train_generator.samples//self.train_generator.batch_size
        self.validation_steps=self.valid_generator.samples//self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list

        )
        self.save_model(
            path=self.config.trained_model_file_path,
            model=self.model
        )

        

In [14]:
try:
        config=ConfigurationManager()
        prepare_callback_config=config.get_prepare_callback()
        prepare_callback=PrepareCallback(prepare_callback_config)
        callback_list=prepare_callback.get_tb_callback

        training_config=config.get_training_config()
        training=Training(config=training_config)
        training.get_base_model()
        training.train_valid_generator()
        training.train(callback_list=callback_list)

except Exception as e:
        raise e

2025-06-03 21:08:14,398 - INFO - config\config.yaml loaded successfully
2025-06-03 21:08:14,403 - INFO - params.yaml loaded successfully
2025-06-03 21:08:14,405 - INFO - Created directory artifacts
2025-06-03 21:08:14,407 - INFO - Created directory artifacts\prepare_callbacks\checkpoint_dir
2025-06-03 21:08:14,408 - INFO - Created directory artifacts\prepare_callbacks\tensorboard_log_dir
2025-06-03 21:08:14,410 - INFO - Created directory artifacts\training
2025-06-03 21:08:14,853 - WARNING - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.


c:\Users\user\Desktop\deep-learning\venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
c:\Users\user\Desktop\deep-learning\venv\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


ValueError: Unknown variable: <Variable path=dense/kernel, shape=(25088, 2), dtype=float32, value=[[ 4.7321320e-03  2.5156662e-03]
 [-2.3566550e-03 -7.8528188e-05]
 [ 1.4194176e-03 -1.2273587e-02]
 ...
 [ 5.8460888e-03 -3.6790920e-03]
 [-1.2415818e-02  2.0771455e-03]
 [-6.8373978e-05  6.2156469e-03]]>. This optimizer can only be called for the variables it was originally built with. When working with a new set of variables, you should recreate a new optimizer instance.